<a href="https://colab.research.google.com/github/enelene/Walmart-Recruiting---Store-Sales-Forecasting/blob/main/notebooks/model_experiment_NBEATS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import sys
# Add the directory containing your script to the Python path
sys.path.append('drive/MyDrive/Colab_Notebooks/Walmart-Recruiting---Store-Sales-Forecasting-main')

In [31]:
!pip install -r  drive/MyDrive/Colab_Notebooks/Walmart-Recruiting---Store-Sales-Forecasting-main/requirements.txt

In [32]:
import pandas as pd
import numpy as np
import os
import warnings
import wandb
import torch
import gc

# NeuralForecast imports
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MAE

In [33]:
def wmae(y_true, y_pred, is_holiday):
    """Computes the Weighted Mean Absolute Error."""
    weights = np.where(np.array(is_holiday, dtype=bool), 5, 1)
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)


In [34]:
wandb.login(key = "720b5644412076fa3e35eb1ffccab9895b8369db")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [35]:
HORIZON = 39
# A common rule of thumb for N-BEATS is to use an input size that is a multiple of the horizon.
INPUT_SIZE =52

In [36]:
try:
    train_df = pd.read_csv('drive/MyDrive/Colab_Notebooks/Walmart-Recruiting---Store-Sales-Forecasting-main/data/train_final.csv', parse_dates=['Date'])
    validation_df = pd.read_csv('drive/MyDrive/Colab_Notebooks/Walmart-Recruiting---Store-Sales-Forecasting-main/data/validation_final.csv', parse_dates=['Date'])
    test_df = pd.read_csv('drive/MyDrive/Colab_Notebooks/Walmart-Recruiting---Store-Sales-Forecasting-main/data/test_final.csv', parse_dates=['Date'])
    print("Successfully loaded all pre-split data files.")
except KeyError:
    print("ERROR: Make sure you have uploaded all three required CSV files.")
    raise

Successfully loaded all pre-split data files.


In [37]:
def prepare_for_nf(df, is_train=True):
    """Prepares a DataFrame for NeuralForecast by renaming and creating necessary columns."""
    df_copy = df.copy()
    # Create the 'unique_id' by combining Store and Dept
    df_copy['unique_id'] = df_copy['Store'].astype(str) + '_' + df_copy['Dept'].astype(str)
    # Rename 'Date' to 'ds'
    df_copy.rename(columns={'Date': 'ds'}, inplace=True)
    # If it's a dataset with a target, rename the target to 'y'
    if is_train and 'Weekly_Sales' in df_copy.columns:
        df_copy.rename(columns={'Weekly_Sales': 'y'}, inplace=True)
    return df_copy

In [38]:

# Prepare the training and validation sets
nf_train_df = prepare_for_nf(train_df)
nf_validation_df = prepare_for_nf(validation_df)

# For evaluation, we need the true 'y' values and holiday flags from the validation set
y_val_true = nf_validation_df['y']
is_holiday_val = nf_validation_df['IsHoliday']

In [39]:
# N-BEATS in its standard implementation only uses future exogenous variables.
futr_exog_list = [
    'IsHoliday', 'Month', 'IsBlackFridayWeek', 'IsLaborDayWeek',
    'WeekOfYear', 'Day', 'IsChristmasWeek', 'IsSuperBowlWeek', 'Year'
]
# Ensure all selected features exist in the dataframe
futr_exog_list = [f for f in futr_exog_list if f in train_df.columns]
print(f"Using {len(futr_exog_list)} future exogenous features for N-BEATS.")


Using 9 future exogenous features for N-BEATS.


In [40]:

# Clear GPU memory before training
gc.collect()
torch.cuda.empty_cache()


In [41]:
from pytorch_lightning.loggers import WandbLogger

In [42]:
wandb.init(project="Walmart-Sales-Forecasting-NF", name="NBEATS-Interpretable-Run-with-Logging", sync_tensorboard=True) # sync_tensorboard is good practice with PL
wandb_logger = WandbLogger(project="Walmart-Sales-Forecasting-NF", name="NBEATS-Interpretable-Run-with-Logging", log_model=False) # log_model=False to avoid logging checkpoints for every run


In [43]:
models = [
    NBEATS(
        h=HORIZON,
        input_size=INPUT_SIZE,
        # Define the interpretable architecture
        stack_types=['trend', 'seasonality', 'identity'],
        n_blocks=[3, 4, 3], # Number of blocks for each stack type
        mlp_units=[[256, 256], [256, 256], [256, 256]], # MLP layer units for each stack
        n_harmonics=2, # Number of fourier terms for seasonality
        n_polynomials=2, # Degree of polynomial for trend
        # General training parameters
        max_steps=1000,
        early_stop_patience_steps=10,
        loss=MAE(),
        scaler_type='robust',
        random_seed=42,
        logger=wandb_logger
    )
]

wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
INFO:lightning_fabric.utilities.seed:Seed set to 42


In [44]:
# The NeuralForecast object manages the training process
nf = NeuralForecast(models=models, freq='W-FRI')

# The fit method takes the training dataframe.
# It will use the last part of the training data for internal validation for early stopping.
nf.fit(df=nf_train_df, val_size=HORIZON)
print("Training complete.")

wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/neuralforecast/core.py:553: UserWarning: Validation set size is larger than the shorter time-series.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/logger

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.


Training complete.


In [45]:
predictions_df = nf.predict()

# Merge predictions with validation data to align for scoring
# We need to ensure the validation set has the same unique_ids as the predictions
validation_ids = predictions_df['unique_id'].unique()
nf_validation_df_filtered = nf_validation_df[nf_validation_df['unique_id'].isin(validation_ids)]

eval_df = pd.merge(nf_validation_df_filtered, predictions_df, on=['unique_id', 'ds'])


INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.


Predicting: |          | 0/? [00:00<?, ?it/s]

In [46]:

# Calculate the WMAE score
wmae_score = wmae(eval_df['y'], eval_df['NBEATS'], eval_df['IsHoliday'])
print(f"Validation WMAE Score: {wmae_score:.4f}")

wandb.log({"validation_wmae": wmae_score})

Validation WMAE Score: 2437.1029


In [47]:
import matplotlib.pyplot as plt
sample_id = '4_1'
sample_eval_df = eval_df[eval_df['unique_id'] == sample_id]
plt.figure(figsize=(10, 6))
plt.plot(sample_eval_df['ds'], sample_eval_df['y'], label='True')
plt.plot(sample_eval_df['ds'], sample_eval_df['NBEATS'], label='Predicted')
plt.title(f'Sample Forecast for {sample_id}')
plt.legend()
wandb.log({"Sample Validation Plot": wandb.Image(plt)})
plt.close() # Close plot to free memory

In [48]:

# First, retrain the model on the combined train + validation data for maximum accuracy
full_train_df = pd.concat([train_df, validation_df])
nf_full_train_df = prepare_for_nf(full_train_df)

In [49]:

# For the final model, we can increase the steps and remove early stopping
final_models = [
    NBEATS(
        h=HORIZON,
        input_size=INPUT_SIZE,
        stack_types=['trend', 'seasonality', 'identity'],
        n_blocks=[3, 4, 3],
        mlp_units=[[256, 256], [256, 256], [256, 256]],
        n_harmonics=2,
        n_polynomials=2,
        max_steps=1500, # Train for a bit longer on the full data
        loss=MAE(),
        scaler_type='robust',
        random_seed=42
    )
]

INFO:lightning_fabric.utilities.seed:Seed set to 42


In [50]:

nf_final = NeuralForecast(models=final_models, freq='W-FRI')
nf_final.fit(df=nf_full_train_df) # No val_size needed here
print("Final model retrained on all available data.")

# Make final predictions
final_predictions_df = nf_final.predict()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 2.4 M  | train
-------------------------------------------------------
2.3 M     Trainable params
28.5 K    Non-trainable params
2.4 M     Total params
9.476     Total estimated model params size (MB)
94        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1500` reached.


Final model retrained on all available data.


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: WARNING When using several event log directories, please call `wandb.tensorboard.patch(root_logdir="...")` before `wandb.init`
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

In [51]:

# Format for submission
test_df['Id'] = test_df['Store'].astype(str) + '_' + test_df['Dept'].astype(str) + '_' + test_df['Date'].dt.strftime('%Y-%m-%d')
final_predictions_df.rename(columns={'ds': 'Date', 'NBEATS': 'Weekly_Sales'}, inplace=True)
final_predictions_df['Date'] = pd.to_datetime(final_predictions_df['Date'])
final_predictions_df['Store'] = final_predictions_df['unique_id'].apply(lambda x: int(x.split('_')[0]))
final_predictions_df['Dept'] = final_predictions_df['unique_id'].apply(lambda x: int(x.split('_')[1]))
prediction_map = final_predictions_df.set_index(['Store', 'Dept', 'Date'])['Weekly_Sales'].to_dict()
test_df['key'] = list(zip(test_df['Store'], test_df['Dept'], test_df['Date']))
test_df['Weekly_Sales'] = test_df['key'].map(prediction_map)

In [52]:

final_submission = test_df[['Id', 'Weekly_Sales']].copy()
final_submission['Weekly_Sales'] = final_submission['Weekly_Sales'].clip(lower=0).fillna(0)
final_submission.to_csv('nf_nbeats_submission.csv', index=False)

print("\nSubmission file 'nf_nbeats_submission.csv' created successfully.")
print(final_submission.head())


Submission file 'nf_nbeats_submission.csv' created successfully.
               Id  Weekly_Sales
0  1_1_2012-11-02  29057.943359
1  1_1_2012-11-09  23176.396484
2  1_1_2012-11-16  22435.439453
3  1_1_2012-11-23  22201.750000
4  1_1_2012-11-30  25912.041016


In [53]:
wandb.init()

epoch,▁▁▁▁▁▂▂▃▃▃▃▃▃▃▄▄▅▅▅▅▁▁▁▂▂▃▃▃▃▄▅▅▅▅▅▇▇▇▇█
global_step,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇██████
hp_metric,▁
ptl/val_loss,█▇▆▄▁▂▄▁▂▃
train_loss_epoch,▅▄▂▂▂▂▁▃▁▂▇██▆█▇▅▃▇▇▃▇▃▆▄
train_loss_step,▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂█▁▂▂▁▁▁▆▁▁▁▂▁▁▁▁▁▁▁▁▁▆▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
valid_loss,█▇▆▄▁▂▄▁▃▃
validation_wmae,▁
version_1/global_step,▁
version_1/hp_metric,▁


In [54]:

# --- 10. Finalize Logging ---
# Log the submission file as a final artifact to our W&B run
print("Logging submission file as a W&B Artifact...")
artifact = wandb.Artifact('submission-file', type='submission')
artifact.add_file('nf_nbeats_submission.csv')
wandb.log_artifact(artifact)

wandb.finish()
print("\nExperiment complete. Check your W&B dashboard for all results and plots.")

Logging submission file as a W&B Artifact...



Experiment complete. Check your W&B dashboard for all results and plots.


In [55]:
import optuna

In [72]:
WANDB_PROJECT = "Walmart-Sales-Forecasting-NF-Tuning"

def objective(trial: optuna.trial.Trial):
    """Optuna objective function to find the best N-BEATSx hyperparameters."""
    gc.collect()
    torch.cuda.empty_cache()

    # Define the search space
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    mlp_units_config = trial.suggest_categorical('mlp_units', [128, 256, 512])
    mlp_units = [[mlp_units_config, mlp_units_config]] * 3
    n_blocks_trend = trial.suggest_int('n_blocks_trend', 1, 3)
    n_blocks_seasonality = trial.suggest_int('n_blocks_seasonality', 1, 4)
    n_blocks_identity = trial.suggest_int('n_blocks_identity', 1, 3)

    # W&B logger for this specific trial
    wandb_logger = WandbLogger(
        project=WANDB_PROJECT,
        name=f"trial-{trial.number}",
        group="NBEATSx-Optuna",
        config=trial.params,
        reinit=True, # Allows re-initialization in the same script
        log_model=False # We only want the final model artifact
    )

    model = NBEATS(
        h=HORIZON,
        input_size=INPUT_SIZE,
        stack_types=['trend', 'seasonality', 'identity'],
        n_blocks=[n_blocks_trend, n_blocks_seasonality, n_blocks_identity],
        mlp_units=mlp_units,
        n_harmonics=2,
        n_polynomials=2,
        learning_rate=learning_rate,
        max_steps=1000, # Increased steps for more reliable tuning
        early_stop_patience_steps=10,
        loss=MAE(),
        scaler_type='robust',
        random_seed=42,
        logger=wandb_logger
    )

    nf = NeuralForecast(models=[model], freq='W-FRI')
    nf.fit(df=nf_train_df, val_size=HORIZON)

    # Evaluation
    predictions_df = nf.predict()
    eval_df = pd.merge(nf_validation_df, predictions_df, on=['unique_id', 'ds'])
    score = wmae(eval_df['y'], eval_df['NBEATS'], eval_df['IsHoliday'])

    wandb_logger.experiment.log({"validation_wmae": score})
    wandb_logger.experiment.finish() # Finish the trial run

    return score

In [73]:
# Create a study and run the optimization
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=15) # Run more trials for a better search


[I 2025-07-31 13:33:49,775] A new study created in memory with name: no-name-7d7b8c46-70c6-4ec2-8ae9-ed5221b34bd0
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
INFO:lightning_fabric.utilities.seed:Seed set to 42
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/usr/local/lib/python3.11/dist-packages/neuralforecast/core.py:553: UserWarning: Validation set size is larger than the shorter time-series.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: Fal

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.
INFO:pytorch_lightning.utilities.rank_zero:Trainer already configured with model summary callbacks: [<class 'pytorch_lightning.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.


Predicting: |          | 0/? [00:00<?, ?it/s]

epoch,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇████
ptl/val_loss,█▅▃▄▁▂▁▁▃▄
train_loss_epoch,█▄▃▃▂▂▁▄▁▄
train_loss_step,▂▁▁▁▁▁▁▁▄▁▁█▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
valid_loss,█▅▃▄▁▂▁▁▃▄
validation_wmae,▁
epoch,9
ptl/val_loss,3205.53247
train_loss_epoch,2.24103
train_loss_step,0.93759


[I 2025-07-31 13:42:15,491] Trial 14 finished with value: 2538.299069611532 and parameters: {'learning_rate': 0.00019449401578119577, 'mlp_units': 256, 'n_blocks_trend': 3, 'n_blocks_seasonality': 3, 'n_blocks_identity': 1}. Best is trial 2 with value: 2348.871846054279.


In [74]:

best_params_raw = study.best_params
best_wmae = study.best_value
print(f"\n--- Tuning Complete ---")
print(f"Best WMAE found: {best_wmae:.4f}")
print(f"Best hyperparameters: {best_params_raw}")


--- Tuning Complete ---
Best WMAE found: 2348.8718
Best hyperparameters: {'learning_rate': 0.0012742717859413285, 'mlp_units': 256, 'n_blocks_trend': 3, 'n_blocks_seasonality': 3, 'n_blocks_identity': 1}


In [78]:
wandb.init(project="Walmart-Sales-Forecasting-NF-Tuning", name="NBEATS-final", sync_tensorboard=True)

In [79]:
print("\n--- Training Final Model with Best Parameters ---")
wandb.config.update(best_params_raw)
wandb.config.update({"input_size": INPUT_SIZE, "horizon": HORIZON})


--- Training Final Model with Best Parameters ---


In [86]:
# Prepare the final model parameters from the Optuna study
final_model_params = {
    'h': HORIZON,
    'input_size': INPUT_SIZE,
    'stack_types': ['trend', 'seasonality', 'identity'],
    'n_blocks': [best_params_raw['n_blocks_trend'], best_params_raw['n_blocks_seasonality'], best_params_raw['n_blocks_identity']],
    # FIX: Format mlp_units correctly as a list of lists for each stack
    'mlp_units': [[best_params_raw['mlp_units'], best_params_raw['mlp_units']]] * len(['trend', 'seasonality', 'identity']),
    'n_harmonics': 2,
    'n_polynomials': 2,
    'learning_rate': best_params_raw['learning_rate'],
    'max_steps': 1500, # Train for longer on the full data
    'loss': MAE(),
    'scaler_type': 'robust',
    'random_seed': 42,
    'logger': wandb_logger
}

In [ ]:
from neuralforecast.losses.pytorch import MAE, MQLoss

In [87]:

# Retrain on full data
full_train_df = pd.concat([train_df, validation_df])
nf_full_train_df = prepare_for_nf(full_train_df)

final_models = [NBEATS(**final_model_params)]
nf_final = NeuralForecast(models=final_models, freq='W-FRI')
nf_final.fit(df=nf_full_train_df)
print("Final model retrained.")

wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
INFO:lightning_fabric.utilities.seed:Seed set to 42
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: Th

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1500` reached.


Final model retrained.


In [89]:
forecast_df

,unique_id,ds,NBEATS
112281,4_1,2012-11-02,56881.593750
112282,4_1,2012-11-09,43297.652344
112283,4_1,2012-11-16,42162.023438
112284,4_1,2012-11-23,41618.398438
112285,4_1,2012-11-30,41967.437500
112286,4_1,2012-12-07,49286.664062
112287,4_1,2012-12-14,58942.953125
112288,4_1,2012-12-21,81618.437500
112289,4_1,2012-12-28,45438.710938
112290,4_1,2013-01-04,35351.398438


In [94]:

# --- 8. Generate Submission and Visualize ---
print("Generating final submission file and visualizations...")
final_predictions_df = nf_final.predict()


Generating final submission file and visualizations...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.


UsageError: Run (1hjg2wdr) is finished. The call to `_config_callback` will be ignored. Please make sure that you are using an active run.

In [96]:
final_predictions_df

,unique_id,Date,Weekly_Sales,Store,Dept
0,10_1,2012-11-02,58958.945312,10,1
1,10_1,2012-11-09,48195.960938,10,1
2,10_1,2012-11-16,44457.261719,10,1
3,10_1,2012-11-23,47952.984375,10,1
4,10_1,2012-11-30,49170.328125,10,1
...,...,...,...,...,...
129904,9_98,2013-01-18,10.967835,9,98
129905,9_98,2013-01-25,12.785383,9,98
129906,9_98,2013-02-01,15.564457,9,98
129907,9_98,2013-02-08,13.061716,9,98


In [91]:

# --- Generate Submission ---
print("Generating final submission file...")
final_predictions_df = nf_final.predict()

# Format for submission
test_df['Id'] = test_df['Store'].astype(str) + '_' + test_df['Dept'].astype(str) + '_' + test_df['Date'].dt.strftime('%Y-%m-%d')
final_predictions_df.rename(columns={'ds': 'Date', 'NBEATS': 'Weekly_Sales'}, inplace=True)
final_predictions_df['Date'] = pd.to_datetime(final_predictions_df['Date'])
final_predictions_df['Store'] = final_predictions_df['unique_id'].apply(lambda x: int(x.split('_')[0]))
final_predictions_df['Dept'] = final_predictions_df['unique_id'].apply(lambda x: int(x.split('_')[1]))
prediction_map = final_predictions_df.set_index(['Store', 'Dept', 'Date'])['Weekly_Sales'].to_dict()
test_df['key'] = list(zip(test_df['Store'], test_df['Dept'], test_df['Date']))
test_df['Weekly_Sales'] = test_df['key'].map(prediction_map)

final_submission = test_df[['Id', 'Weekly_Sales']].copy()
final_submission['Weekly_Sales'] = final_submission['Weekly_Sales'].clip(lower=0).fillna(0)
final_submission.to_csv('nf_nbeats_submission_tuned.csv', index=False)
print("\nSubmission file 'nf_nbeats_submission_tuned.csv' created successfully.")


Generating final submission file...


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
wandb: WARNING `wandb.require('service')` is a no-op as it is now the default behavior.


Predicting: |          | 0/? [00:00<?, ?it/s]


Submission file 'nf_nbeats_submission_tuned.csv' created successfully.


In [92]:

# Log final score and artifact
wandb.log({"final_validation_wmae": best_wmae})
artifact = wandb.Artifact('submission-file-nbeats-tuned', type='submission')
artifact.add_file('nf_nbeats_submission_tuned.csv')
wandb.log_artifact(artifact)
wandb.finish()

epoch,▁▁▁▁▁▂▂▃▃▁▁▁▁▂▂▃▃▃▃▃▄▄▅▅▅▅▅▅▅▅▆▆▇▇▇▇▇▇▇█
final_validation_wmae,▁
global_step,▁▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▇▇▇▇██
hp_metric,▁
train_loss_epoch,▇██▇██▆█▆▄▁▆▇▂▆▁▅▃
train_loss_step,▂█▁▁▁▂▁▁▂█▁▁▁▂▁▁▁▁▆▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▆▂▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
epoch,14
final_validation_wmae,2348.87185
global_step,399
hp_metric,-1
